In [ ]:
import requests
from bs4 import BeautifulSoup
import urllib
import pandas as pd
import numpy as np
import json
from time import sleep, perf_counter as pf
import itertools as it

In [ ]:
# Temperature, pop, sunrise, sunset
temp = pd.read_csv('datasets/weather/temperature.csv')#[['datetime', 'Vancouver', 'Denver']]

# 3 Column Vancouver
t_van = temp[['datetime', 'Vancouver']]
t_van.insert(1, 'City', 'Vancouver')
t_van.columns=['Datetime', 'City', 'Temperature']

# 3 Column Denver
t_den = temp[['datetime', 'Denver']]
t_den.insert(1, 'City', 'Denver')
t_den.columns=['Datetime', 'City', 'Temperature']
temp = pd.concat([t_van, t_den])

temp.reset_index(drop=True, inplace=True)
display(temp)

In [ ]:
desc = pd.read_csv('datasets/weather/weather_description.csv')#[['Datetime', 'Vancouver', 'Denver']]

# 3 Column Vancouver
d_van = desc[['datetime', 'Vancouver']]
d_van.insert(1, 'City', 'Vancouver')
d_van.columns=['Datetime', 'City', 'Weather']

# 3 Column Denver
d_den = desc[['datetime', 'Denver']]
d_den.insert(1, 'City', 'Denver')
d_den.columns=['Datetime', 'City', 'Weather']
desc = pd.concat([d_van, d_den])

desc.reset_index(drop=True, inplace=True)
display(desc)

In [ ]:
weather = pd.merge(temp, desc, left_on=['Datetime', 'City'], right_on=['Datetime', 'City'])
display(weather)

In [ ]:
datetime = pd.to_datetime(weather['Datetime'])
weather['Date'] = [d.date() for d in datetime]
#weather['TIME'] = [d.time() for d in datetime]
weather.drop('Datetime', axis=1, inplace=True)
display(weather)

In [ ]:
# Average out over the day
output_lst = []
for date in weather.Date.unique():
    for city in weather.City.unique():
        df = weather.loc[(weather['Date'] == date) & (weather['City'] == city)]
        output_lst.append({
            'City': city,
            'Temperature': df.Temperature.mean(),
            'Weather': df.Weather.mode()[0] if len(df.Weather.mode()) > 0 else np.nan,
            'Date': date
        })
weather = pd.DataFrame(output_lst)
display(weather)

In [ ]:
s_van = pd.read_csv('datasets/sunrise_sunset_denver.csv')
s_van['City'] = 'Vancouver'
s_den = pd.read_csv('datasets/sunrise_sunset_denver.csv')
s_den['City'] = 'Denver'
sun = pd.concat([s_van, s_den])
datetime = pd.to_datetime(sun[['Year', 'Month', 'Day']])
sun['Date'] = [d.date() for d in datetime]
sun = sun[['Date', 'City', 'Sunrise', 'Sunset']]
display(sun)

In [ ]:
full = pd.merge(weather, sun, left_on=['Date', 'City'], right_on=['Date', 'City'])
full.insert(0, 'Weather_key', full.index)
display(full)

In [ ]:
full.to_csv('datasets/weather.csv', index=False)

In [ ]:
final = full.drop(['City', 'Date'], axis=1)
final.to_csv('datasets/final/weather.csv')
display(final)